In [0]:
configs = {
    "fs.azure.account.auth.type": "OAuth",
    "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
    "fs.azure.account.oauth2.client.id": "client-id",
    "fs.azure.account.oauth2.client.secret": "client-secret-id",
    "fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/tenant-id/oauth2/token",
}

dbutils.fs.mount(
    source="abfss://formula-one-data@dataformula1.dfs.core.windows.net/raw-data", #containername@storagename
    mount_point="/mnt/dataformula1",
    extra_configs=configs
)


Out[6]: True

In [0]:
%fs
ls "/mnt/dataformula1"

path,name,size,modificationTime
dbfs:/mnt/dataformula1/deliveries.csv,deliveries.csv,18235327,1724501015000
dbfs:/mnt/dataformula1/formula-one-data/,formula-one-data/,0,0
dbfs:/mnt/dataformula1/matches.csv,matches.csv,143378,1724501001000
dbfs:/mnt/dataformula1/most_runs_average_strikerate.csv,most_runs_average_strikerate.csv,23917,1724500999000


In [0]:
df_matches = spark.read.option("header", "true").csv("/mnt/dataformula1/formula-one-data/raw-data/matches.csv")

In [0]:
df_matches.show(10)

+---+--------+---------+----------+--------------------+--------------------+--------------------+-------------+------+----------+--------------------+-----------+--------------+---------------+--------------------+--------------+-------------+-------+
| id|  Season|     city|      date|               team1|               team2|         toss_winner|toss_decision|result|dl_applied|              winner|win_by_runs|win_by_wickets|player_of_match|               venue|       umpire1|      umpire2|umpire3|
+---+--------+---------+----------+--------------------+--------------------+--------------------+-------------+------+----------+--------------------+-----------+--------------+---------------+--------------------+--------------+-------------+-------+
|  1|IPL-2017|Hyderabad|05-04-2017| Sunrisers Hyderabad|Royal Challengers...|Royal Challengers...|        field|normal|         0| Sunrisers Hyderabad|         35|             0|   Yuvraj Singh|Rajiv Gandhi Inte...|   AY Dandekar|     NJ Llo

In [0]:
df_most_runs_average_strikerate = spark.read.option("header", "true").csv("/mnt/dataformula1/formula-one-data/raw-data/most_runs_average_strikerate.csv")
df_most_runs_average_strikerate.show()

+--------------+----------+---+-------------+------------------+------------------+
|       batsman|total_runs|out|numberofballs|           average|        strikerate|
+--------------+----------+---+-------------+------------------+------------------+
|       V Kohli|      5426|152|         4111| 35.69736842105263|131.98735100948676|
|      SK Raina|      5386|160|         3916|           33.6625|137.53830439223697|
|     RG Sharma|      4902|161|         3742|  30.4472049689441|130.99946552645645|
|     DA Warner|      4717|114|         3292| 41.37719298245614|143.28675577156744|
|      S Dhawan|      4601|137|         3665|33.583941605839414|125.53888130968622|
|      CH Gayle|      4525|110|         2972| 41.13636363636363| 152.2543741588156|
|      MS Dhoni|      4450|118|         3206| 37.71186440677966|138.80224578914536|
|    RV Uthappa|      4420|156|         3381|28.333333333333332|130.73055309080155|
|AB de Villiers|      4414|104|         2902| 42.44230769230769|152.10199862

In [0]:
spark


SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.3.2 
 Master 
 local[*, 4] 
 AppName 
 Databricks Shell

In [0]:
# Filter the DataFrame where strikerate is greater than 125
filtered_df = df_most_runs_average_strikerate.filter(df_most_runs_average_strikerate.strikerate > 135)

# Show the filtered DataFrame to verify (optional)
filtered_df.show()

+--------------+----------+---+-------------+------------------+------------------+
|       batsman|total_runs|out|numberofballs|           average|        strikerate|
+--------------+----------+---+-------------+------------------+------------------+
|      SK Raina|      5386|160|         3916|           33.6625|137.53830439223697|
|     DA Warner|      4717|114|         3292| 41.37719298245614|143.28675577156744|
|      CH Gayle|      4525|110|         2972| 41.13636363636363| 152.2543741588156|
|      MS Dhoni|      4450|118|         3206| 37.71186440677966|138.80224578914536|
|AB de Villiers|      4414|104|         2902| 42.44230769230769|152.10199862164023|
|     SR Watson|      3590|115|         2566|31.217391304347824|139.90646921278253|
|     YK Pathan|      3222|110|         2240| 29.29090909090909|143.83928571428572|
|    KA Pollard|      2772|102|         1879|27.176470588235293|147.52527940393827|
|      V Sehwag|      2728| 98|         1746|27.836734693877553|156.24284077

In [0]:
df_most_runs_average_strikerate.count()

Out[15]: 516

In [0]:
# Define the exact path to the existing 'transformed-data' directory
output_path = "/mnt/dataformula1/formula-one-data/transformed-data/filtered_strikerates"

# Write the DataFrame to CSV without additional subdirectories
filtered_df.write.mode('overwrite').option("header", "true").csv(output_path)



In [0]:
from pyspark.sql.functions import col

filtered_df = filtered_df.withColumn("average", col("average").cast("float"))

# Sort the DataFrame by the 'average' column in ascending order
sorted_df = filtered_df.orderBy("average", ascending=False)

# Show the sorted DataFrame
sorted_df.show()

+--------------+----------+---+-------------+---------+------------------+
|       batsman|total_runs|out|numberofballs|  average|        strikerate|
+--------------+----------+---+-------------+---------+------------------+
|    J Bairstow|       459|  8|          284|   57.375|161.61971830985914|
|       HM Amla|       577| 13|          407|44.384617|141.76904176904176|
|AB de Villiers|      4414|104|         2902|42.442307|152.10199862164023|
|     DA Warner|      4717|114|         3292|41.377193|143.28675577156744|
|      CH Gayle|      4525|110|         2972|41.136364| 152.2543741588156|
|     ML Hayden|      1107| 27|          802|     41.0| 138.0299251870324|
|      KL Rahul|      1988| 49|         1428| 40.57143| 139.2156862745098|
|    JC Buttler|      1396| 37|          919| 37.72973|151.90424374319912|
|      MS Dhoni|      4450|118|         3206|37.711864|138.80224578914536|
|       RR Pant|      1767| 49|         1071|36.061226| 164.9859943977591|
|      AN Ahmed|        3

In [0]:
from pyspark.sql.functions import col, format_number

# Assuming the DataFrame 'sorted_df' is already loaded
# Convert the 'strikerate' and 'average' columns to float and format them to two decimal places
df_final = sorted_df \
    .withColumn("strikerate", col("strikerate").cast("float")) \
    .withColumn("strikerate", format_number("strikerate", 2)) \
    .withColumn("average", col("average").cast("float")) \
    .withColumn("average", format_number("average", 2))

# Show the DataFrame to verify the formatting
df_final.show()


+--------------+----------+---+-------------+-------+----------+
|       batsman|total_runs|out|numberofballs|average|strikerate|
+--------------+----------+---+-------------+-------+----------+
|    J Bairstow|       459|  8|          284|  57.38|    161.62|
|       HM Amla|       577| 13|          407|  44.38|    141.77|
|AB de Villiers|      4414|104|         2902|  42.44|    152.10|
|     DA Warner|      4717|114|         3292|  41.38|    143.29|
|      CH Gayle|      4525|110|         2972|  41.14|    152.25|
|     ML Hayden|      1107| 27|          802|  41.00|    138.03|
|      KL Rahul|      1988| 49|         1428|  40.57|    139.22|
|    JC Buttler|      1396| 37|          919|  37.73|    151.90|
|      MS Dhoni|      4450|118|         3206|  37.71|    138.80|
|       RR Pant|      1767| 49|         1071|  36.06|    164.99|
|      AN Ahmed|        36|  1|           26|  36.00|    138.46|
|       CA Lynn|      1288| 37|          909|  34.81|    141.69|
|    AD Russell|      141

In [0]:
df_final.count()

Out[33]: 91

In [0]:
# Specify the output path under the 'transformed-data' directory
output_path = "/mnt/dataformula1/formula-one-data/transformed-data/filtered_stats"
# Write the DataFrame to CSV in the specified path
df_final.write.mode('overwrite').option("header", "true").csv(output_path)
